Let us compare the fine-tuned T5 and CondBERT models on the validation set.

Loading the models:

In [6]:
import pandas as pd
import numpy as np
import sys

sys.path.insert(0, '../')

from src.models.condbert.condbert_inference import CondBERT
from src.models.t5_paraphrase.t5_inference import T5Paraphrase
from src.toxicity_metric import measure_toxicity

In [2]:
condbert = CondBERT(dir_path='../src/models/condbert/')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
t5 = T5Paraphrase(model_dir_path='../models/t5-paraphrase/')

Some weights of the model checkpoint at SkolkovoInstitute/roberta_toxicity_classifier were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This

Reading the data:

In [4]:
df = pd.read_csv('../data/interim/validation.tsv', delimiter='\t', index_col=0)
df.head()

,reference,translation,similarity,lenght_diff,ref_tox,trn_tox,source,target
372253,We got the goods when we killed the cop.,it's over when we killed that cop today.,0.728086,0.000000,0.234415,0.998152,it's over when we killed that cop today.,We got the goods when we killed the cop.
573874,That murderous tweaker Max in exchange for tha...,that sparking son-of-a-bitch Max in exchange f...,0.799426,0.094488,0.006455,0.997834,that sparking son-of-a-bitch Max in exchange f...,That murderous tweaker Max in exchange for tha...
131143,The man's a fool! How can I possibly tell who ...,how am I supposed to know who the message is w...,0.724833,0.161290,0.999650,0.000040,The man's a fool! How can I possibly tell who ...,how am I supposed to know who the message is w...
293461,That's a pretty fucked up way to die.,that's a hell of a way to die.,0.819554,0.184211,0.997765,0.039029,That's a pretty fucked up way to die.,that's a hell of a way to die.
82325,I get it. Maybe that's what elliot's doing.,that's what Elliot might be trying to do.,0.689799,0.045455,0.952854,0.000048,I get it. Maybe that's what elliot's doing.,that's what Elliot might be trying to do.


Inference on the validation dataset:

In [5]:
# t5_paraphrase
df['t5'] = df['source'].apply(lambda s: t5.detoxify(sentence=s))

In [7]:
# condbert
df['condbert'] = df['source'].apply(lambda s: condbert.detoxify(sentence=s))

In [3]:
df = pd.read_csv('../data/interim/predictions.tsv', delimiter='\t', index_col=0)
df.head()

,reference,translation,similarity,lenght_diff,ref_tox,trn_tox,source,target,t5,condbert
372253,We got the goods when we killed the cop.,it's over when we killed that cop today.,0.728086,0.000000,0.234415,0.998152,it's over when we killed that cop today.,We got the goods when we killed the cop.,It's over when the cop was killed today.,it ' s over when we lost that cop today .
573874,That murderous tweaker Max in exchange for tha...,that sparking son-of-a-bitch Max in exchange f...,0.799426,0.094488,0.006455,0.997834,that sparking son-of-a-bitch Max in exchange f...,That murderous tweaker Max in exchange for tha...,That sparking son-in-law Max in exchange for s...,"that sparking brother - of - a - , max in exch..."
131143,The man's a fool! How can I possibly tell who ...,how am I supposed to know who the message is w...,0.724833,0.161290,0.999650,0.000040,The man's a fool! How can I possibly tell who ...,how am I supposed to know who the message is w...,how can I tell who the message is until I know...,the man ' s a false ! how can i possibly tell ...
293461,That's a pretty fucked up way to die.,that's a hell of a way to die.,0.819554,0.184211,0.997765,0.039029,That's a pretty fucked up way to die.,that's a hell of a way to die.,that's a pretty terrible way to die.,that ' s a pretty done up way to be .
82325,I get it. Maybe that's what elliot's doing.,that's what Elliot might be trying to do.,0.689799,0.045455,0.952854,0.000048,I get it. Maybe that's what elliot's doing.,that's what Elliot might be trying to do.,maybe that's what Elliot's doing.,i get it . maybe that ' s what elliot ' s doing .


Now, let's measure the average toxicity level, and toxicity percentage for the `source`, `target`, `t5`, and `condbert` columns:

In [11]:
results_dict = {
    'toxicity_percentage': {},
    'avg_toxicity': {}
}

for column in ['source', 'target', 't5', 'condbert']:
    toxicity_labels, toxicity_scores = measure_toxicity(df[column].to_list())
    
    results_dict['toxicity_percentage'][column] = np.sum(toxicity_labels) / len(toxicity_labels)
    results_dict['avg_toxicity'][column] = np.mean(toxicity_scores)

Some weights of the model checkpoint at SkolkovoInstitute/roberta_toxicity_classifier were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 32/32 [01:38<00:00,  3.06s/it]
Some weights of the model checkpoint at SkolkovoInstitute/roberta_toxicity_classifier were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you a

In [12]:
df_results = pd.DataFrame.from_dict(results_dict)
df_results

,toxicity_percentage,avg_toxicity
source,0.830,-3.414778
target,0.087,5.573105
t5,0.103,6.222356
condbert,0.066,6.672241


As you can see, the condbert model is the best one according to the introduced metrics :)

However, we have no fluency metric which is also an important one when talking about text seq2seq task. Condbert's generations often look clumsy and distort the initial message meaning.
So, if I have managed to implement some fluency metric, the condbert would certainly perform poorly with it.

We see that t5 generations on average has less toxicity compared to the `target` column (6.22 > 5.57). However, t5's toxicity percentage is slightly greater than the target's one.